# Week 11: Integration - Practice Notebook---**Date**: 2025-11-22  **Course**: BSMA1001 - Mathematics for Data Science I  **Level**: Foundation  **Week**: 11 of 12  **Topic Area**: Calculus - Integration---## Learning ObjectivesThis notebook provides hands-on practice with:- Visualizing definite integrals as areas- Computing Riemann sum approximations- Applying the Fundamental Theorem of Calculus- Using integration techniques (substitution, parts)- Computing probabilities from PDFs- Finding expected values of distributions- Numerical integration methods## Prerequisites```pythonimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy import integratefrom scipy.stats import uniform, norm, expon```

In [ ]:
import numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy import integratefrom scipy.stats import uniform, norm, exponimport warningswarnings.filterwarnings('ignore')# Set random seednp.random.seed(42)# Configure plottingplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print("✓ Libraries imported successfully")print(f"NumPy version: {np.__version__}")print(f"SciPy version: {integrate.__version__}")

## 1. Visualizing Integration via Riemann SumsDefinite integrals are defined as limits of Riemann sums:$$\int_a^b f(x) \, dx = \lim_{n \to \infty} \sum_{i=1}^{n} f(x_i^*) \Delta x$$Let's visualize how Riemann sums approximate the area under a curve.

In [ ]:
def riemann_sum_visual(f, a, b, n, method='right'):    """Visualize Riemann sum approximation.        Args:        f: Function to integrate        a, b: Integration limits        n: Number of rectangles        method: 'left', 'right', or 'midpoint'    """    x = np.linspace(a, b, 1000)    y = f(x)        # Compute Riemann sum    dx = (b - a) / n    xi = np.linspace(a, b, n+1)        if method == 'left':        sample_points = xi[:-1]    elif method == 'right':        sample_points = xi[1:]    else:  # midpoint        sample_points = (xi[:-1] + xi[1:]) / 2        riemann_sum = sum(f(sample_points) * dx)        # True value    true_value, _ = integrate.quad(f, a, b)    error = abs(riemann_sum - true_value)        # Plot    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))        # Left: Rectangles    ax1.plot(x, y, 'b-', linewidth=2.5, label='f(x)', zorder=10)        for i in range(n):        x_rect = [xi[i], xi[i+1], xi[i+1], xi[i], xi[i]]        y_rect = [0, 0, f(sample_points[i]), f(sample_points[i]), 0]        ax1.fill(x_rect, y_rect, alpha=0.3, edgecolor='red', linewidth=1.5)        ax1.plot(sample_points[i], f(sample_points[i]), 'ro', markersize=6, zorder=5)        ax1.fill_between(x, 0, y, alpha=0.15, color='blue', label='True area')    ax1.set_xlabel('x', fontsize=12)    ax1.set_ylabel('f(x)', fontsize=12)    ax1.set_title(f'{method.capitalize()} Riemann Sum (n={n})',                   fontsize=13, fontweight='bold')    ax1.legend(fontsize=11)    ax1.grid(True, alpha=0.3)        # Right: Information    ax2.axis('off')    info_text = f"""    Riemann Sum Approximation    {'='*50}        Method: {method.capitalize()} endpoint    Interval: [{a}, {b}]    Number of rectangles: n = {n}    Width of each rectangle: Δx = {dx:.4f}        Riemann Sum: {riemann_sum:.6f}    True Integral: {true_value:.6f}    Error: {error:.6f}    Relative Error: {100*error/abs(true_value):.2f}%        Formula:    ∫[{a},{b}] f(x)dx ≈ Σ f(x_i) Δx        As n → ∞, approximation → true value    """        ax2.text(0.1, 0.5, info_text, fontsize=11, family='monospace',             verticalalignment='center',             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))        plt.tight_layout()    plt.show()        return riemann_sum, true_value# Example: f(x) = x^2 on [0, 2]f = lambda x: x**2print("Comparing Riemann sum approximations:\n")print("=" * 70)for n in [4, 10, 50]:    riemann, true = riemann_sum_visual(f, 0, 2, n, method='right')    print(f"n={n:>3}: Riemann={riemann:.6f}, True={true:.6f}, Error={abs(riemann-true):.6f}")    print()

## 2. Comparing Riemann Sum MethodsDifferent sample points give different approximations:- **Left endpoint:** Uses $f(x_i)$ where $x_i$ is left edge- **Right endpoint:** Uses $f(x_{i+1})$ where $x_{i+1}$ is right edge  - **Midpoint:** Uses $f((x_i + x_{i+1})/2)$ (usually most accurate)

In [ ]:
def compare_riemann_methods(f, a, b, n_values):    """Compare accuracy of different Riemann sum methods."""    true_value, _ = integrate.quad(f, a, b)        methods = ['left', 'right', 'midpoint']    results = {method: [] for method in methods}        for n in n_values:        dx = (b - a) / n        xi = np.linspace(a, b, n+1)                # Left endpoint        sample_points = xi[:-1]        results['left'].append(sum(f(sample_points) * dx))                # Right endpoint        sample_points = xi[1:]        results['right'].append(sum(f(sample_points) * dx))                # Midpoint        sample_points = (xi[:-1] + xi[1:]) / 2        results['midpoint'].append(sum(f(sample_points) * dx))        # Plot convergence    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))        colors = {'left': 'red', 'right': 'blue', 'midpoint': 'green'}        # Approximation values    for method in methods:        ax1.plot(n_values, results[method], 'o-', color=colors[method],                linewidth=2, markersize=8, label=method.capitalize())        ax1.axhline(y=true_value, color='black', linestyle='--',                linewidth=2, label=f'True value = {true_value:.4f}')    ax1.set_xlabel('Number of rectangles (n)', fontsize=12)    ax1.set_ylabel('Approximation', fontsize=12)    ax1.set_title('Convergence of Riemann Sums', fontsize=13, fontweight='bold')    ax1.legend(fontsize=11)    ax1.grid(True, alpha=0.3)        # Error (log scale)    for method in methods:        errors = [abs(val - true_value) for val in results[method]]        ax2.loglog(n_values, errors, 'o-', color=colors[method],                  linewidth=2, markersize=8, label=method.capitalize())        ax2.set_xlabel('Number of rectangles (n)', fontsize=12)    ax2.set_ylabel('Absolute Error', fontsize=12)    ax2.set_title('Error vs n (log-log scale)', fontsize=13, fontweight='bold')    ax2.legend(fontsize=11)    ax2.grid(True, alpha=0.3, which='both')        plt.tight_layout()    plt.show()        # Print table    print("\nRiemann Sum Comparison:")    print("=" * 90)    print(f"{'n':<8} {'Left':<15} {'Right':<15} {'Midpoint':<15} {'True Value':<15}")    print("=" * 90)        for i, n in enumerate(n_values):        print(f"{n:<8} {results['left'][i]:<15.6f} {results['right'][i]:<15.6f} "              f"{results['midpoint'][i]:<15.6f} {true_value:<15.6f}")        print("=" * 90)# Test with f(x) = x^2 on [0, 2]f = lambda x: x**2n_values = [5, 10, 20, 50, 100]compare_riemann_methods(f, 0, 2, n_values)print("\nObservation: Midpoint rule typically has smallest error!")

## 3. Fundamental Theorem of Calculus**FTC Part 1:** If $F(x) = \int_a^x f(t) \, dt$, then $F'(x) = f(x)$**FTC Part 2:** $\int_a^b f(x) \, dx = F(b) - F(a)$ where $F' = f$Let's visualize how the accumulation function $F(x)$ relates to $f(x)$.

In [ ]:
def visualize_ftc(f, f_name, a, b):    """Visualize Fundamental Theorem of Calculus."""    x_vals = np.linspace(a, b, 1000)    f_vals = f(x_vals)        # Compute accumulation function F(x) = ∫_a^x f(t) dt    F_vals = []    for x in x_vals:        F_x, _ = integrate.quad(f, a, x)        F_vals.append(F_x)    F_vals = np.array(F_vals)        # Numerical derivative of F (should equal f)    F_prime_numerical = np.gradient(F_vals, x_vals)        fig, axes = plt.subplots(2, 2, figsize=(14, 10))        # Plot 1: Original function f(x)    ax = axes[0, 0]    ax.plot(x_vals, f_vals, 'b-', linewidth=2.5)    ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('f(x)', fontsize=12)    ax.set_title(f'Original Function: {f_name}', fontsize=13, fontweight='bold')    ax.grid(True, alpha=0.3)    ax.axhline(y=0, color='k', linewidth=0.5)        # Plot 2: Accumulation function F(x)    ax = axes[0, 1]    ax.plot(x_vals, F_vals, 'r-', linewidth=2.5)    ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('F(x)', fontsize=12)    ax.set_title(f'Accumulation: F(x) = ∫_{a}^x {f_name} dt',                  fontsize=13, fontweight='bold')    ax.grid(True, alpha=0.3)    ax.axhline(y=0, color='k', linewidth=0.5)        # Plot 3: F'(x) vs f(x)    ax = axes[1, 0]    ax.plot(x_vals, f_vals, 'b-', linewidth=2.5, label='f(x)', alpha=0.7)    ax.plot(x_vals, F_prime_numerical, 'r--', linewidth=2, label="F'(x) (numerical)", alpha=0.7)    ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('y', fontsize=12)    ax.set_title("FTC Part 1: F'(x) = f(x)", fontsize=13, fontweight='bold')    ax.legend(fontsize=11)    ax.grid(True, alpha=0.3)        # Plot 4: Area interpretation    ax = axes[1, 1]    ax.plot(x_vals, f_vals, 'b-', linewidth=2.5)        # Shade area from a to specific x value    x_highlight = a + 0.6 * (b - a)    x_shade = x_vals[x_vals <= x_highlight]    y_shade = f(x_shade)    ax.fill_between(x_shade, 0, y_shade, alpha=0.3, color='green',                     label=f'Area = F({x_highlight:.2f})')    ax.axvline(x=x_highlight, color='green', linestyle='--', linewidth=2)        area_value, _ = integrate.quad(f, a, x_highlight)    ax.text(x_highlight, max(f_vals)*0.5, f'F({x_highlight:.2f})\n= {area_value:.3f}',           fontsize=11, bbox=dict(boxstyle='round', facecolor='lightgreen'))        ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('f(x)', fontsize=12)    ax.set_title('Area Accumulation', fontsize=13, fontweight='bold')    ax.legend(fontsize=11)    ax.grid(True, alpha=0.3)    ax.axhline(y=0, color='k', linewidth=0.5)        plt.tight_layout()    plt.show()# Example 1: f(x) = x^2visualize_ftc(lambda x: x**2, 'x²', 0, 3)# Example 2: f(x) = sin(x)visualize_ftc(lambda x: np.sin(x), 'sin(x)', 0, 2*np.pi)

## 4. Computing Integrals with FTC Part 2$$\int_a^b f(x) \, dx = F(b) - F(a)$$where $F$ is any antiderivative of $f$.

In [ ]:
def compute_integral(f, F, a, b, description):    """Compute integral using FTC and verify numerically."""    # Analytical using FTC    analytical = F(b) - F(a)        # Numerical using SciPy    numerical, error = integrate.quad(f, a, b)        print(f"\n{description}")    print("=" * 70)    print(f"∫[{a},{b}] f(x) dx")    print(f"\nAnalytical (FTC): F({b}) - F({a}) = {F(b):.6f} - {F(a):.6f} = {analytical:.6f}")    print(f"Numerical (scipy):  {numerical:.6f}")    print(f"Match: {np.isclose(analytical, numerical)} (diff: {abs(analytical-numerical):.2e})")        return analytical, numericalprint("Computing Definite Integrals:")print("=" * 70)# Example 1: ∫_0^2 x^2 dx = [x^3/3]_0^2 = 8/3compute_integral(    f=lambda x: x**2,    F=lambda x: x**3 / 3,    a=0, b=2,    description="Example 1: ∫₀² x² dx = [x³/3]₀²")# Example 2: ∫_0^π sin(x) dx = [-cos(x)]_0^π = 2compute_integral(    f=lambda x: np.sin(x),    F=lambda x: -np.cos(x),    a=0, b=np.pi,    description="Example 2: ∫₀^π sin(x) dx = [-cos(x)]₀^π")# Example 3: ∫_1^e 1/x dx = [ln(x)]_1^e = 1compute_integral(    f=lambda x: 1/x,    F=lambda x: np.log(x),    a=1, b=np.e,    description="Example 3: ∫₁^e (1/x) dx = [ln(x)]₁^e")# Example 4: ∫_0^1 e^x dx = [e^x]_0^1 = e - 1compute_integral(    f=lambda x: np.exp(x),    F=lambda x: np.exp(x),    a=0, b=1,    description="Example 4: ∫₀¹ eˣ dx = [eˣ]₀¹")print("\n" + "=" * 70)print("\nConclusion: FTC provides exact values; numerical methods approximate!")

## 5. Integration by Substitution**Formula:** $\int f(g(x)) g'(x) \, dx = \int f(u) \, du$ where $u = g(x)$**For definite integrals:** Change the limits!$$\int_a^b f(g(x)) g'(x) \, dx = \int_{g(a)}^{g(b)} f(u) \, du$$

In [ ]:
print("Integration by Substitution Examples:")print("=" * 70)# Example 1: ∫ 2x cos(x²) dxprint("\nExample 1: ∫ 2x cos(x²) dx")print("-" * 70)print("Let u = x², then du = 2x dx")print("∫ 2x cos(x²) dx = ∫ cos(u) du = sin(u) + C = sin(x²) + C")# Verifyf1 = lambda x: 2*x * np.cos(x**2)F1 = lambda x: np.sin(x**2)  # Antiderivativea, b = 0, 1analytical = F1(b) - F1(a)numerical, _ = integrate.quad(f1, a, b)print(f"\nVerification on [0, 1]:")print(f"Analytical: {analytical:.6f}")print(f"Numerical:  {numerical:.6f}")print(f"Match: {np.isclose(analytical, numerical)}")# Example 2: ∫₀¹ x e^(x²) dxprint("\n" + "=" * 70)print("\nExample 2: ∫₀¹ x e^(x²) dx")print("-" * 70)print("Let u = x², then du = 2x dx, so x dx = (1/2)du")print("When x=0: u=0; when x=1: u=1")print("∫₀¹ x e^(x²) dx = (1/2) ∫₀¹ e^u du = (1/2)[e^u]₀¹")print("             = (1/2)(e - 1)")f2 = lambda x: x * np.exp(x**2)analytical = (np.e - 1) / 2numerical, _ = integrate.quad(f2, 0, 1)print(f"\nAnalytical: (e-1)/2 = {analytical:.6f}")print(f"Numerical:  {numerical:.6f}")print(f"Match: {np.isclose(analytical, numerical)}")# Example 3: ∫ (2x+1)/(x²+x+1) dxprint("\n" + "=" * 70)print("\nExample 3: ∫ (2x+1)/(x²+x+1) dx")print("-" * 70)print("Notice: derivative of denominator = 2x+1")print("Let u = x²+x+1, then du = (2x+1) dx")print("∫ (2x+1)/(x²+x+1) dx = ∫ (1/u) du = ln|u| + C = ln|x²+x+1| + C")f3 = lambda x: (2*x + 1) / (x**2 + x + 1)F3 = lambda x: np.log(x**2 + x + 1)a, b = 0, 2analytical = F3(b) - F3(a)numerical, _ = integrate.quad(f3, a, b)print(f"\nVerification on [0, 2]:")print(f"Analytical: ln(7) - ln(1) = {analytical:.6f}")print(f"Numerical:  {numerical:.6f}")print(f"Match: {np.isclose(analytical, numerical)}")

## 6. Integration in Probability**Probability Density Function (PDF):** $f(x) \geq 0$ and $\int_{-\infty}^{\infty} f(x) \, dx = 1$**Probability:** $P(a \leq X \leq b) = \int_a^b f(x) \, dx$**Expected Value:** $E[X] = \int_{-\infty}^{\infty} x f(x) \, dx$

In [ ]:
# Example 1: Uniform distribution on [0, 1]print("Example 1: Uniform Distribution U[0,1]")print("=" * 70)print("PDF: f(x) = 1 for 0 ≤ x ≤ 1")print()# Verify normalizationtotal_prob, _ = integrate.quad(lambda x: 1, 0, 1)print(f"Total probability: ∫₀¹ 1 dx = {total_prob}")# Compute P(0.3 ≤ X ≤ 0.7)prob, _ = integrate.quad(lambda x: 1, 0.3, 0.7)print(f"P(0.3 ≤ X ≤ 0.7) = ∫₀.₃^₀.₇ 1 dx = {prob}")# Expected valueexpected, _ = integrate.quad(lambda x: x * 1, 0, 1)print(f"E[X] = ∫₀¹ x·1 dx = {expected}")# Visualizex = np.linspace(-0.5, 1.5, 1000)pdf = np.where((x >= 0) & (x <= 1), 1, 0)fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# PDFax1.plot(x, pdf, 'b-', linewidth=2.5)ax1.fill_between(x, 0, pdf, where=((x >= 0.3) & (x <= 0.7)),                  alpha=0.3, color='green', label='P(0.3 ≤ X ≤ 0.7)')ax1.set_xlabel('x', fontsize=12)ax1.set_ylabel('f(x)', fontsize=12)ax1.set_title('Uniform[0,1] PDF', fontsize=13, fontweight='bold')ax1.legend(fontsize=11)ax1.grid(True, alpha=0.3)ax1.set_ylim(-0.1, 1.3)# CDF: F(x) = ∫_{-∞}^x f(t) dtcdf = np.where(x < 0, 0, np.where(x <= 1, x, 1))ax2.plot(x, cdf, 'r-', linewidth=2.5)ax2.set_xlabel('x', fontsize=12)ax2.set_ylabel('F(x)', fontsize=12)ax2.set_title('Cumulative Distribution Function', fontsize=13, fontweight='bold')ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n" + "=" * 70)# Example 2: Exponential distributionprint("\nExample 2: Exponential Distribution (λ=1)")print("=" * 70)print("PDF: f(x) = λe^(-λx) for x ≥ 0")print()lam = 1f_exp = lambda x: lam * np.exp(-lam * x)# Verify normalizationtotal_prob, _ = integrate.quad(f_exp, 0, np.inf)print(f"Total probability: ∫₀^∞ λe^(-λx) dx = {total_prob:.6f}")# P(X ≤ 2)prob, _ = integrate.quad(f_exp, 0, 2)print(f"P(X ≤ 2) = ∫₀² e^(-x) dx = {prob:.6f}")print(f"Using CDF: 1 - e^(-2) = {1 - np.exp(-2):.6f}")# Expected valueexpected, _ = integrate.quad(lambda x: x * f_exp(x), 0, np.inf)print(f"E[X] = ∫₀^∞ x·λe^(-λx) dx = {expected:.6f}")print(f"Theoretical: 1/λ = {1/lam}")# Visualizex = np.linspace(0, 5, 1000)pdf_exp = expon.pdf(x, scale=1/lam)fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# PDFax1.plot(x, pdf_exp, 'b-', linewidth=2.5)ax1.fill_between(x, 0, pdf_exp, where=(x <= 2), alpha=0.3,                  color='green', label='P(X ≤ 2)')ax1.axvline(x=expected, color='red', linestyle='--', linewidth=2,            label=f'E[X]={expected:.2f}')ax1.set_xlabel('x', fontsize=12)ax1.set_ylabel('f(x)', fontsize=12)ax1.set_title('Exponential(λ=1) PDF', fontsize=13, fontweight='bold')ax1.legend(fontsize=11)ax1.grid(True, alpha=0.3)# CDFcdf_exp = expon.cdf(x, scale=1/lam)ax2.plot(x, cdf_exp, 'r-', linewidth=2.5)ax2.set_xlabel('x', fontsize=12)ax2.set_ylabel('F(x)', fontsize=12)ax2.set_title('Exponential CDF', fontsize=13, fontweight='bold')ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\nKey insight: Integration connects PDFs to probabilities!")

## 7. Area Between CurvesArea between $y = f(x)$ and $y = g(x)$ from $x = a$ to $x = b$:$$\text{Area} = \int_a^b |f(x) - g(x)| \, dx$$If $f(x) \geq g(x)$, then Area $= \int_a^b [f(x) - g(x)] \, dx$

In [ ]:
def area_between_curves(f, g, a, b, f_name='f(x)', g_name='g(x)'):    """Compute and visualize area between two curves."""    # Find where f ≥ g and where g > f    x = np.linspace(a, b, 1000)    y_f = f(x)    y_g = g(x)        # Compute area    area, _ = integrate.quad(lambda x: abs(f(x) - g(x)), a, b)        # Plot    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))        # Left: Functions    ax1.plot(x, y_f, 'b-', linewidth=2.5, label=f_name)    ax1.plot(x, y_g, 'r-', linewidth=2.5, label=g_name)    ax1.fill_between(x, y_f, y_g, where=(y_f >= y_g), alpha=0.3,                      color='green', label='Area')    ax1.fill_between(x, y_f, y_g, where=(y_g > y_f), alpha=0.3, color='orange')    ax1.set_xlabel('x', fontsize=12)    ax1.set_ylabel('y', fontsize=12)    ax1.set_title(f'Area Between Curves = {area:.4f}', fontsize=13, fontweight='bold')    ax1.legend(fontsize=11)    ax1.grid(True, alpha=0.3)        # Right: Difference function    diff = y_f - y_g    ax2.plot(x, diff, 'purple', linewidth=2.5, label='f(x) - g(x)')    ax2.fill_between(x, 0, diff, where=(diff >= 0), alpha=0.3,                      color='green', label='f ≥ g')    ax2.fill_between(x, 0, diff, where=(diff < 0), alpha=0.3,                      color='orange', label='g > f')    ax2.axhline(y=0, color='k', linewidth=1)    ax2.set_xlabel('x', fontsize=12)    ax2.set_ylabel('f(x) - g(x)', fontsize=12)    ax2.set_title('Difference Function', fontsize=13, fontweight='bold')    ax2.legend(fontsize=11)    ax2.grid(True, alpha=0.3)        plt.tight_layout()    plt.show()        return area# Example 1: Area between x² and x from 0 to 1print("Example 1: Area between y = x² and y = x on [0, 1]")print("=" * 70)f1 = lambda x: xg1 = lambda x: x**2area1 = area_between_curves(f1, g1, 0, 1, 'y=x', 'y=x²')print(f"\nAnalytical: ∫₀¹ (x - x²) dx = [x²/2 - x³/3]₀¹ = 1/2 - 1/3 = 1/6")print(f"Computed: {area1:.6f}")print(f"Exact: {1/6:.6f}")# Example 2: Area between sin(x) and cos(x)print("\n" + "=" * 70)print("\nExample 2: Area between sin(x) and cos(x) on [0, π/2]")f2 = lambda x: np.cos(x)g2 = lambda x: np.sin(x)area2 = area_between_curves(f2, g2, 0, np.pi/2, 'cos(x)', 'sin(x)')print(f"\nComputed area: {area2:.6f}")

## 8. Numerical Integration MethodsWhen antiderivatives are difficult or impossible to find, use numerical methods:- **Trapezoidal Rule:** Linear approximation- **Simpson's Rule:** Parabolic approximation- **Monte Carlo:** Random sampling

In [ ]:
def compare_numerical_methods(f, a, b, true_value, n_values):    """Compare different numerical integration methods."""    results = {'trapezoidal': [], 'simpsons': [], 'quad': []}        for n in n_values:        x = np.linspace(a, b, n+1)        y = f(x)                # Trapezoidal rule        trap = np.trapz(y, x)        results['trapezoidal'].append(trap)                # Simpson's rule (requires odd number of points, i.e., even n)        if n % 2 == 0:            simp = integrate.simps(y, x)            results['simpsons'].append(simp)        else:            results['simpsons'].append(np.nan)                # scipy.integrate.quad (adaptive)        quad_result, _ = integrate.quad(f, a, b)        results['quad'].append(quad_result)        # Plot    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))        # Approximations    ax1.plot(n_values, results['trapezoidal'], 'ro-', label='Trapezoidal', linewidth=2, markersize=8)    valid_simp = [(n, v) for n, v in zip(n_values, results['simpsons']) if not np.isnan(v)]    if valid_simp:        n_simp, v_simp = zip(*valid_simp)        ax1.plot(n_simp, v_simp, 'bo-', label="Simpson's", linewidth=2, markersize=8)    ax1.axhline(y=true_value, color='black', linestyle='--', linewidth=2, label=f'True = {true_value:.6f}')    ax1.set_xlabel('Number of points (n+1)', fontsize=12)    ax1.set_ylabel('Approximation', fontsize=12)    ax1.set_title('Numerical Integration Convergence', fontsize=13, fontweight='bold')    ax1.legend(fontsize=11)    ax1.grid(True, alpha=0.3)        # Errors (log scale)    trap_errors = [abs(v - true_value) for v in results['trapezoidal']]    ax2.loglog(n_values, trap_errors, 'ro-', label='Trapezoidal', linewidth=2, markersize=8)        if valid_simp:        simp_errors = [abs(v - true_value) for v in v_simp]        ax2.loglog(n_simp, simp_errors, 'bo-', label="Simpson's", linewidth=2, markersize=8)        ax2.set_xlabel('Number of points (n+1)', fontsize=12)    ax2.set_ylabel('Absolute Error', fontsize=12)    ax2.set_title('Error vs Number of Points (log-log)', fontsize=13, fontweight='bold')    ax2.legend(fontsize=11)    ax2.grid(True, alpha=0.3, which='both')        plt.tight_layout()    plt.show()        # Print table    print("\nNumerical Integration Comparison:")    print("=" * 90)    print(f"{'n':<6} {'Trapezoidal':<18} {'Simpson':<18} {'Error (Trap)':<15} {'Error (Simp)':<15}")    print("=" * 90)        for i, n in enumerate(n_values):        trap_err = abs(results['trapezoidal'][i] - true_value)        simp_val = results['simpsons'][i]        if not np.isnan(simp_val):            simp_err = abs(simp_val - true_value)            print(f"{n:<6} {results['trapezoidal'][i]:<18.10f} {simp_val:<18.10f} "                  f"{trap_err:<15.2e} {simp_err:<15.2e}")        else:            print(f"{n:<6} {results['trapezoidal'][i]:<18.10f} {'N/A':<18} "                  f"{trap_err:<15.2e} {'N/A':<15}")        print("=" * 90)    print(f"True value: {true_value}")# Example: ∫₀¹ e^(-x²) dx (Gaussian integral, no elementary antiderivative!)f = lambda x: np.exp(-x**2)true_value, _ = integrate.quad(f, 0, 1)print("Example: ∫₀¹ e^(-x²) dx (no elementary antiderivative)")print("=" * 70)n_values = [4, 8, 16, 32, 64]compare_numerical_methods(f, 0, 1, true_value, n_values)print(f"\nObservation: Simpson's rule converges faster than trapezoidal!")

## 9. Practice ProblemsSolve computationally:**Problem 1:** Approximate $\int_0^2 (x^3 - 2x + 1) \, dx$ using Riemann sum (n=10, right endpoints). Compare with exact value.**Problem 2:** Compute $\int_1^3 \frac{1}{x^2} \, dx$ using FTC. Verify numerically.**Problem 3:** For uniform distribution on [2, 5], find $P(3 \leq X \leq 4)$ and $E[X]$.**Problem 4:** Find area between $y = e^x$ and $y = x^2$ from $x = 0$ to $x = 1$.**Problem 5:** Use Simpson's rule with n=10 to approximate $\int_0^{\pi} \sin^2(x) \, dx$.

In [ ]:
print("=" * 70)print("PRACTICE PROBLEM SOLUTIONS")print("=" * 70)# Problem 1print("\nProblem 1: Riemann sum for ∫₀² (x³ - 2x + 1) dx")f1 = lambda x: x**3 - 2*x + 1a, b, n = 0, 2, 10dx = (b - a) / nxi = np.linspace(a, b, n+1)riemann = sum(f1(xi[1:]) * dx)F1 = lambda x: x**4/4 - x**2 + xexact = F1(b) - F1(a)print(f"  Riemann sum (n={n}, right): {riemann:.6f}")print(f"  Exact (FTC): [x⁴/4 - x² + x]₀² = {exact:.6f}")print(f"  Error: {abs(riemann - exact):.6f}")# Problem 2print("\n" + "=" * 70)print("\nProblem 2: ∫₁³ (1/x²) dx")f2 = lambda x: 1 / x**2F2 = lambda x: -1 / xexact2 = F2(3) - F2(1)numerical2, _ = integrate.quad(f2, 1, 3)print(f"  Analytical: [-1/x]₁³ = -1/3 - (-1) = {exact2:.6f}")print(f"  Numerical:  {numerical2:.6f}")print(f"  Match: {np.isclose(exact2, numerical2)}")# Problem 3print("\n" + "=" * 70)print("\nProblem 3: Uniform[2, 5]")# PDF: f(x) = 1/(5-2) = 1/3 for 2 ≤ x ≤ 5pdf3 = lambda x: 1/3prob3, _ = integrate.quad(pdf3, 3, 4)expected3, _ = integrate.quad(lambda x: x * pdf3(x), 2, 5)print(f"  PDF: f(x) = 1/3 for x ∈ [2, 5]")print(f"  P(3 ≤ X ≤ 4) = ∫₃⁴ (1/3) dx = {prob3:.6f}")print(f"  Expected: {prob3} = {1/3:.6f}")print(f"  E[X] = ∫₂⁵ x·(1/3) dx = {expected3:.6f}")print(f"  Theoretical: (2+5)/2 = {3.5}")# Problem 4print("\n" + "=" * 70)print("\nProblem 4: Area between e^x and x² on [0, 1]")f4a = lambda x: np.exp(x)f4b = lambda x: x**2area4, _ = integrate.quad(lambda x: f4a(x) - f4b(x), 0, 1)print(f"  Area = ∫₀¹ (e^x - x²) dx")print(f"       = [e^x - x³/3]₀¹")print(f"       = (e - 1/3) - (1 - 0)")print(f"       = e - 4/3")print(f"  Computed: {area4:.6f}")print(f"  Exact: {np.e - 4/3:.6f}")# Problem 5print("\n" + "=" * 70)print("\nProblem 5: Simpson's rule for ∫₀^π sin²(x) dx")f5 = lambda x: np.sin(x)**2x5 = np.linspace(0, np.pi, 11)  # n=10 requires 11 pointsy5 = f5(x5)simpsons5 = integrate.simps(y5, x5)# Exact: ∫ sin²(x) dx = x/2 - sin(2x)/4 + CF5 = lambda x: x/2 - np.sin(2*x)/4exact5 = F5(np.pi) - F5(0)print(f"  Simpson's rule (n=10): {simpsons5:.6f}")print(f"  Exact: [x/2 - sin(2x)/4]₀^π = π/2 = {exact5:.6f}")print(f"  Error: {abs(simpsons5 - exact5):.2e}")print("\n" + "=" * 70)

## 10. Self-Assessment ChecklistCheck your understanding:**Integration Fundamentals:**- [ ] Understand integration as reverse of differentiation- [ ] Can compute antiderivatives using basic rules- [ ] Remember to include constant of integration ($+C$)- [ ] Understand definite integrals as limits of Riemann sums- [ ] Can visualize integrals as areas under curves**Fundamental Theorem of Calculus:**- [ ] Know FTC Part 1: $\frac{d}{dx}\left[\int_a^x f(t)dt\right] = f(x)$- [ ] Know FTC Part 2: $\int_a^b f(x)dx = F(b) - F(a)$- [ ] Can use FTC to evaluate definite integrals- [ ] Understand connection between derivatives and integrals**Integration Techniques:**- [ ] Can apply substitution (u-substitution) method- [ ] Remember to change limits for definite integrals with substitution- [ ] Can use integration by parts: $\int u\,dv = uv - \int v\,du$- [ ] Know LIATE rule for choosing $u$ in integration by parts**Applications:**- [ ] Can compute area under a curve- [ ] Can find area between two curves- [ ] Can calculate average value of a function- [ ] Understand probability density functions (PDFs)- [ ] Can compute probabilities using integration- [ ] Can find expected values of random variables**Numerical Integration:**- [ ] Understand when numerical methods are needed- [ ] Can apply trapezoidal rule- [ ] Can apply Simpson's rule- [ ] Know that Simpson's rule is more accurate than trapezoidal---## Next Steps**Week 12 Preview: Comprehensive Applications**- Review and synthesis of Weeks 4-11- Multivariate calculus introduction (partial derivatives, gradients)- Optimization algorithms (gradient descent, Newton's method)- Differential equations basics- End-to-end data science applications---**Excellent work! Integration completes your calculus foundation. You now have the mathematical tools for optimization, probability, and machine learning. Week 12 will synthesize everything into practical applications!**